In [2]:
from tensorflow import keras
from tensorflow.keras.datasets import reuters
import numpy as np


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 


max_words = 10000
maxlen = 200
emb_dim = 50


In [3]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words = max_words)

In [4]:
x_train.shape, x_test.shape

((8982,), (2246,))

In [5]:
lengths = np.array([len(x) for x in x_train])
print(np.mean(lengths), np.median(lengths), np.min(lengths), np.max(lengths))

145.5398574927633 95.0 13 2376


- 로이터 뉴스 카테고리 분류 데이터
- 총 11258개의 뉴스 기사 데이터
    - 46개의 카테고리 분류
    - 예)
        - 중부 지방은 대체로 맑겠으나 , 나무 지방은 구름이 많겠습니다 => 날씨
        - 올 초 부터 유동성의 힘으로 주가가 일정하게 상승했습니다 -> 주식

In [6]:
np.unique(y_train, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]),
 array([  55,  432,   74, 3159, 1949,   17,   48,   16,  139,  101,  124,
         390,   49,  172,   26,   20,  444,   39,   66,  549,  269,  100,
          15,   41,   62,   92,   24,   15,   48,   19,   45,   39,   32,
          11,   50,   10,   49,   19,   19,   24,   36,   30,   13,   21,
          12,   18]))

# 포켓몬 배틀

In [7]:
# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(
    x_train,
    y_train, 
    test_size = 0.2,
    stratify = y_train,
    random_state = 34
)

In [8]:
# 패딩 
train_seq = pad_sequences(x_train, maxlen = maxlen)
val_seq = pad_sequences(x_val, maxlen = maxlen)

In [9]:
# LSTM 모델 생성
model = keras.Sequential()

model.add(keras.Input(shape = (maxlen,))) # 200개정도 슥~ input 에 넣어주고
model.add(keras.layers.Embedding(max_words, emb_dim))

model.add(keras.layers.LSTM(16, dropout = 0.3, return_sequences=True, go_backwards= True)) # 이것이 LSTM
model.add(keras.layers.GRU(32, dropout =0.3, return_sequences=True, go_backwards= True)) # GRU
model.add(keras.layers.LSTM(16, dropout = 0.3, return_sequences=True, go_backwards= True)) # 이것이 LSTM
model.add(keras.layers.GRU(32, dropout = 0.4)) # 이것이 LSTM

# 46개의 범주
model.add(keras.layers.Dense(92, activation="relu"))
model.add(keras.layers.Dense(46, activation="softmax"))

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 50)             │         500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 200, 16)             │           4,288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 200, 32)             │           4,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 200, 16)             │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 32)                  │           4,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 92)                  │           3,036 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 46)                  │           4,278 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 524,338 (2.00 MB)

 Trainable params: 524,338 (2.00 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate= 1e-4)
model.compile(
    optimizer= rmsprop, 
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

es_cb = keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)

history = model.fit(
    train_seq, y_train, epochs = 300,
    batch_size = 64, validation_data = (val_seq, y_val), callbacks = [es_cb]
)

Epoch 1/300
113/113 ━━━━━━━━━━━━━━━━━━━━ 20s 133ms/step - accuracy: 0.3193 - loss: 3.7379 - val_accuracy: 0.3517 - val_loss: 3.5042
Epoch 2/300
113/113 ━━━━━━━━━━━━━━━━━━━━ 15s 128ms/step - accuracy: 0.3513 - loss: 3.0915 - val_accuracy: 0.3517 - val_loss: 2.6015
Epoch 3/300
 44/113 ━━━━━━━━━━━━━━━━━━━━ 8s 125ms/step - accuracy: 0.3498 - loss: 2.6536 